# Primer Job 
Escribe en el archivo format para cada jugador su puntaje promedio y su puntaje historico nuevo y anterior (en la primera vuelta son iguales) y si alguien lo reto

In [464]:
from MRE import Job
partidasDir = "./partidas/"
promedioDir = "../punto2/contabilizado/"
outputDir = "./format/"

In [465]:
def fmapBase(key, value, context):
    lista = value.split()
    ph = 1
    if lista[0] == '0' and lista[1] == '0':
        promedio = 1/1
    else:
        promedio = (int(lista[0])  )/(int(lista[1])) 
    context.write((int(key), "PP"), (promedio,ph))

def fmapPartida(retador, value, context):
    lista = value.split("\t")
    retado = lista[0]
    context.write((int(retado), "SENT TO"), int(retador))

def cmpShort(key1,key2):
    if(key1[0] == key2[0]):
        return 0
    elif(key1[0] < key2[0]):
        return -1
    else:
        return 1
    
def cmpShuffle(key1, key2):
    if(key1[1] == key2[1]):
        return 0
    elif(key1[1] == "PP"): #PP y PH es el primero 
        return -1
    else:
        return 1 #Caso Contribucion
    
def fred(key, values, context):
    estructura = next(values) 
    # Estructura esperada: [PP_i , PH_viejo]
    print(key, estructura)
    pp_i = float(estructura[0])
    ph = estructura[1]
    
    
    for v in values: 
        print(key, v)
        retador = v
        context.write(key[0], (pp_i, puntaje_heroico_inicial, ph, retador))

    
puntaje_heroico_inicial = 1

jobCount = Job(promedioDir, outputDir, fmapBase, fred)
jobCount.setParams(puntaje_heroico_inicial)
jobCount.addInputPath(partidasDir, fmapPartida)
jobCount.setShuffleCmp(cmpShort)
jobCount.setSortCmp(cmpShuffle)

success = jobCount.waitForCompletion()

print(success)

(1, 'SENT TO') (1919.2222222222222, 1)
(1, 'SENT TO') 84
(1, 'SENT TO') 59
(1, 'SENT TO') 77
(1, 'SENT TO') 56
(1, 'SENT TO') 13
(1, 'SENT TO') 28
(1, 'SENT TO') 29
(1, 'SENT TO') 98
(1, 'SENT TO') 37
(1, 'SENT TO') 89
(1, 'SENT TO') 40
(1, 'SENT TO') 67
(1, 'SENT TO') 20
(1, 'SENT TO') 25
(2, 'SENT TO') (1.0, 1)
(2, 'SENT TO') 1
(2, 'SENT TO') 90
(3, 'PP') (2524.5, 1)
(4, 'SENT TO') (2663.0, 1)
(4, 'SENT TO') 67
(4, 'SENT TO') 90
(4, 'SENT TO') 43
(5, 'PP') (3288.0, 1)
(5, 'PP') 4
(5, 'PP') 45
(6, 'SENT TO') (1.0, 1)
(6, 'SENT TO') 89
(6, 'SENT TO') 73
(7, 'PP') (1231.5, 1)
(8, 'SENT TO') (2060.25, 1)
(8, 'SENT TO') 28
(8, 'SENT TO') 41
(8, 'SENT TO') 10
(9, 'SENT TO') (2468.0, 1)
(9, 'SENT TO') 57
(9, 'SENT TO') 23
(10, 'SENT TO') (1325.6666666666667, 1)
(10, 'SENT TO') 1
(10, 'SENT TO') 58
(11, 'SENT TO') (1411.0, 1)
(11, 'SENT TO') 25
(11, 'SENT TO') 71
(12, 'PP') (2886.3333333333335, 1)
(12, 'PP') 77
(13, 'SENT TO') (3858.6666666666665, 1)
(13, 'SENT TO') 52
(14, 'SENT TO') (2518.

# Segundo job  = iterador
Toma la salida del job anterior para saber los datos de cada jugador contribuir al calculo de quienes lo retaron (si nadie lo reto no envia info)

In [466]:

def fmapJugador(key, value, context):
    lista = value.split("\t")
    promedio = lista[0]
    ph_inicial = lista[1]
    ph_viejo = lista[2]
    context.write((int(key), "DATOS"), (promedio, ph_inicial, ph_viejo))
    #print(f"escribi {key}, {lista}")
    

    # A -> B  === PPA, PPB, PHB
    # A DATOS = 1.5 0.8
    # B DATOS = 0.7 0.4
    # A CONTRIBUCION = 0.7 0.4

def mapJugador(key, value, context):
    lista = value.split("\t")
    promedio = float(lista[0])
    ph_inicial = float(lista[1])
    if len(lista) > 3:  # Evitar casos sin retadores (este caso se da si un jugador no fue retado por a nadie)
        retador = lista[3]
        context.write((int(retador), "CONTRIBUCION"), (promedio, ph_inicial))

def cmpShort(key1,key2):
    if(key1[0] == key2[0]):
        return 0
    elif(key1[0] < key2[0]):
        return -1
    else:
        return 1
    
def cmpShuffle(key1, key2):
    if(key1[1] == key2[1]):
        return 0
    elif(key1[1] == "DATOS"): #PP y PH es el primero 
        return -1
    else:
        return 1 #Caso Contribucion

def fred(key, values, context):
    estructura = next(values) 
    # Estructura esperada: [PP_i , PH_viejo]
    #print(f"lei: {key}, {estructura}")
    print(key)
    pp_i = float(estructura[0])
    ph_viejo = float(estructura[1])
    dato = estructura[2]
    
    acumulador = 0
    for v in values:
        # v viene del Mapper como (PP_j, PH_j)
        pp_j = float(v[0])  
        ph_j = float(v[1])
        acumulador += ph_j * pp_i / pp_j
    
    nuevo_ph = (alfa * acumulador) + (1 - alfa)
    
    # Emitimos manteniendo la estructura para la siguiente iteración
    # Formato: ID \t PP \t PH_Nuevo \t PH_Viejo
    context.write(key[0], (pp_i, nuevo_ph, ph_viejo))

# JOB INTERMEDIO QUE SOLO CALCULA EL MAE ENTRE DOS ITERACIONES
def map_mae(key, value, context):
    # La estructura de value es: [PP, PH_Nuevo, PH_Viejo]
    datos = value.split("\t")
        
    ph_nuevo = float(datos[1])
    ph_viejo = float(datos[2])
    
    # Calculamos el error absoluto: |PH_nuevo - PH_viejo|
    error_absoluto = abs(ph_nuevo - ph_viejo)
    
    # Enviamos a una clave fija para que un Reducer calcule el promedio final
    # Emitimos (error, 1) para poder calcular el promedio (Suma / Total)
    context.write("GLOBAL_MAE", (error_absoluto, 1))

def red_mae(key, values, context):
    suma_error_absoluto = 0
    conteo_jugadores = 0
    
    for err, count in values:
        suma_error_absoluto += err
        conteo_jugadores += count
    # MAE = Suma de errores absolutos / N
    mae_final = suma_error_absoluto / conteo_jugadores if conteo_jugadores > 0 else 0
    
    context.write("RESULTADO_MAE", mae_final)


# JOB PRINCIPAL DE ITERACIÓN
def fmapBase(key, value, context):
    lista = value.split()   
    context.write((int(key), "PP"), (float(lista[0]), float(lista[1]), float(lista[2])))

def fmapPartida(retador, value, context):
    lista = value.split("\t")
    retado = lista[0]
    context.write((int(retado), "SENT TO"), int(retador))

def cmpShort(key1,key2):
    if(key1[0] == key2[0]):
        return 0
    elif(key1[0] < key2[0]):
        return -1
    else:
        return 1
    
def cmpShuffle(key1, key2):
    if(key1[1] == key2[1]):
        return 0
    elif(key1[1] == "PP"): #PP y PH es el primero 
        return -1
    else:
        return 1 #Caso Contribucion
    
def fredBase(key, values, context):
    estructura = next(values) 
    # Estructura esperada: [PP_i , PH_viejo]
    #if key[0] == 12:
    #    print(key,estructura)
    
    pp_i = float(estructura[0])
    ph = estructura[1]
    ph_viejo = estructura[2]
   
    for v in values: 
        retador = v
        #print(key, v)
        context.write(key[0], (pp_i, ph, ph_viejo, retador))

# Parámetros iniciales
alfa = 0.1
error_tolerado = 0.001
max_iter = 50
input_dir = "./format/" # Salida del Job anterior (ID \t [PP, PH, {Retados}])

for i in range(max_iter):
    output_dir = f"./iteraciones/iteracion_{i}/"
    
    # Configuramos el Job
    job = Job(input_dir, output_dir, fmapJugador, fred)
    job.addInputPath(input_dir, mapJugador)
    job.setParams(alfa)
    job.setShuffleCmp(cmpShort)
    job.setSortCmp(cmpShuffle)

    success = job.waitForCompletion()
    
    # --- CÁLCULO DE CONVERGENCIA (MSE) ---
    mae_dir = f"./mae/mae_iter_{i}/"
    
    # 1. Ejecutar Job de cálculo de PH (el que ya tienes)
    # 2. Ejecutar Job de MAE
    jobMAE = Job(output_dir, mae_dir, map_mae, red_mae)
    jobMAE.waitForCompletion()
    
    # 3. Leer el resultado del MAE desde el Driver
    # (El driver solo lee un archivo de texto de pocos bytes)
    try:
        with open(f"{mae_dir}/output.txt", "r") as f:
            # Formato esperado: RESULTADO_MAE \t 0.00045
            linea = f.read().split("\t")
            mae_actual = float(linea[1])
            
        print(f">>> Iteración {i} - MAE: {mae_actual:.6f}")
        
        if mae_actual < error_tolerado:
            print(f"Convergió en iteración {i} con MAE {mae_actual}")
            break
            
    except Exception as e:
        print(f"Error leyendo MAE: {e}")
        break
    
    # JOB DE JOIN ENTRE PARTIDAS Y JUGADORES PARA LA SIGUIENTE ITERACIÓN
    intermedio_dir = f"./intermedios/intermedio_iter_{i}/"
    jobJoin = Job(output_dir, intermedio_dir, fmapBase, fredBase)
    jobJoin.addInputPath(partidasDir, fmapPartida)
    jobJoin.setShuffleCmp(cmpShort)
    jobJoin.setSortCmp(cmpShuffle)

    success = jobJoin.waitForCompletion()
    
    # El output de esta vuelta es el input de la siguiente
    input_dir = intermedio_dir

print(success)



(1, 'DATOS')
(2, 'DATOS')
(3, 'CONTRIBUCION')


IndexError: tuple index out of range

Tercer Job saca top 10 

In [ ]:
inputDir = "./format/"
outputDir = "./out/"

In [ ]:
import heapq

def fmap(key, value, context):
    lista = value.split("\t")
    context.write(1, (key, float(lista[-1])))

def fcombiner(key, values, context):
    # simple combiner: sum counts locally
    top_maximo = []
    
    for v in values:
        elemento = (v[1],v[0])
        if len(top_maximo) < 10:
            # Si hay menos de 10 elementos, simplemente lo añadimos al heap.
            heapq.heappush(top_maximo, elemento)
        else:
            # Si el nuevo score (elemento[0]) es MAYOR que el elemento MÁS PEQUEÑO 
            # del heap (top_maximo[0]), reemplazamos el pequeño.
            if elemento[0] > top_maximo[0][0]:
                heapq.heapreplace(top_maximo, elemento)
    top_10_final = sorted(top_maximo, key=lambda e: e[0], reverse=True)
    for (maximo, clave) in top_10_final:
        context.write(key, (clave,maximo))

def fred(key, values, context):
    # simple combiner: sum counts locally
    top_maximo = []
    
    for v in values:
        elemento = (v[1],v[0])
        if len(top_maximo) < 10:
            # Si hay menos de 10 elementos, simplemente lo añadimos al heap.
            heapq.heappush(top_maximo, elemento)
        else:
            # Si el nuevo score (elemento[0]) es MAYOR que el elemento MÁS PEQUEÑO 
            # del heap (top_maximo[0]), reemplazamos el pequeño.
            if elemento[0] > top_maximo[0][0]:
                heapq.heapreplace(top_maximo, elemento)
    top_10_final = sorted(top_maximo, key=lambda e: e[0], reverse=True)
    for (maximo, clave) in top_10_final:
        context.write(clave, maximo)

jobCountBoth = Job(inputDir, outputDir, fmap, fred)
jobCountBoth.setCombiner(fcombiner)
success = jobCountBoth.waitForCompletion()

print(success)

True
